Would it be possible for datashader to check user input early, rather than failing with a variety of different errors later on during computation?

This notebook shows a variety of error messages that come during various computations, all caused by what I think are invalid inputs (i.e. inputs that are obviously incorrect at the start, before any computation).

In [1]:
import datashader as ds
import datashader.transfer_functions as tf
import pandas as pd
import numpy as np
import xarray as xr

In [2]:
ds.__version__

'0.6.5'

In [3]:
num=1000
np.random.seed(1)

dists = {cat: pd.DataFrame.from_items([('x',np.random.normal(x,s,num)), 
                                       ('y',np.random.normal(y,s,num)), 
                                       ('val',val), 
                                       ('cat',cat)])      
         for x,  y,  s,  val, cat in 
         [(  2,  2, 0.03, 10, "d1"), 
          (  2, -2, 0.10, 20, "d2"), 
          ( -2, -2, 0.50, 30, "d3"), 
          ( -2,  2, 1.00, 40, "d4"), 
          (  0,  0, 3.00, 50, "d5")] }

df = pd.concat(dists,ignore_index=True)
df["cat"]=df["cat"].astype("category")

In [4]:
# would be nicer to get an error here...
canvas = ds.Canvas(plot_width=-1, plot_height=-1)

In [5]:
# ...than here?
tf.shade(canvas.points(df,'x','y'))

ValueError: negative dimensions are not allowed

In [6]:
xs = np.arange(10)[::-1]
ys = np.arange(5)[::-1]
arr = xs*ys[np.newaxis].T
xarr = xr.DataArray(arr, coords={'X': xs, 'Y': ys}, dims=['Y', 'X'])

In [7]:
xarr

<xarray.DataArray (Y: 5, X: 10)>
array([[36, 32, 28, 24, 20, 16, 12,  8,  4,  0],
       [27, 24, 21, 18, 15, 12,  9,  6,  3,  0],
       [18, 16, 14, 12, 10,  8,  6,  4,  2,  0],
       [ 9,  8,  7,  6,  5,  4,  3,  2,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
Coordinates:
  * X        (X) int64 9 8 7 6 5 4 3 2 1 0
  * Y        (Y) int64 4 3 2 1 0

In [8]:
# ok

In [9]:
cvs = ds.Canvas(plot_width=10, plot_height=5, x_range=(-.5, 9.5), y_range=(-.5, 4.5))

In [10]:
cvs.raster(xarr)

<xarray.DataArray (Y: 5, X: 10)>
array([[36, 32, 28, 24, 20, 16, 12,  8,  4,  0],
       [27, 24, 21, 18, 15, 12,  9,  6,  3,  0],
       [18, 16, 14, 12, 10,  8,  6,  4,  2,  0],
       [ 9,  8,  7,  6,  5,  4,  3,  2,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
Coordinates:
  * X        (X) float64 9.0 8.0 7.0 6.0 5.0 4.0 3.0 2.0 1.0 0.0
  * Y        (Y) float64 4.0 3.0 2.0 1.0 0.0
Attributes:
    res:      -1.0

In [11]:
# invalid plot_width

In [12]:
cvs = ds.Canvas(plot_width=-10, plot_height=5, x_range=(-.5, 9.5), y_range=(-.5, 4.5))

In [13]:
cvs.raster(xarr)

ValueError: negative dimensions are not allowed

In [14]:
# questionable plot_width

In [15]:
cvs = ds.Canvas(plot_width=0, plot_height=5, x_range=(-.5, 9.5), y_range=(-.5, 4.5))

In [16]:
cvs.raster(xarr)

ZeroDivisionError: division by zero

In [17]:
# invalid x_range

In [18]:
cvs = ds.Canvas(plot_width=10, plot_height=5, x_range=(-.5, -1), y_range=(-.5, 4.5))

In [19]:
cvs.raster(xarr)

KeyError: 'Failed at nopython (convert to parfors)\n442'

In [20]:
## questionable x_range

In [21]:
cvs = ds.Canvas(plot_width=10, plot_height=5, x_range=(-.5, -.5), y_range=(-.5, 4.5))

In [22]:
cvs.raster(xarr)

ZeroDivisionError: float division by zero